# Разметка данных

# Пакеты

In [1]:
import pandas as pd
from enum import Enum
from datasets import Dataset, DatasetDict

In [4]:
df = pd.read_csv("places_address_elements.csv")
df = df.fillna('')

In [5]:
df

,region_name,region_type_name,region_type_name_short,area_name,area_type_name,area_type_name_short,city_name,city_type_name,city_type_name_short,territory_name,territory_type_name,territory_type_name_short,street_name,street_type_name,street_type_name_short
0,Адыгея,Республика,Респ,,,,,,,,,,,,
1,Башкортостан,Республика,Респ,,,,,,,,,,,,
2,Алтай,Республика,Респ,,,,,,,,,,,,
3,Кабардино-Балкарская,Республика,Респ,,,,,,,,,,,,
4,Калмыкия,Республика,Респ,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519314,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Малиновая,Улица,ул.
1519315,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Персиковая,Улица,ул.
1519316,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Яблоневая,Улица,ул.
1519317,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Садовая,Улица,ул.


# Словарь ElementType 

In [6]:

class ElementType(Enum):
    OTHER = 0
    REGION=1
    REGION_TYPE=2
    AREA=3
    AREA_TYPE=4
    TERRITORY=5
    TERRITORY_TYPE=6
    CITY=7
    CITY_TYPE=8
    STREET=9
    STREET_TYPE=10


In [21]:
e = ElementType.OTHER
e.value

0

# Представление адреса в виде типизированных слов-токенов

Разбиение адреса на слова-токены:

In [7]:
str ='Алтайский Край, Залесовский Район, Залесово Село, автодорога Мартыново-Тогул-Залесово Территория, 151-й Километр'
tokens = str.split(' ')
tokens

['Алтайский',
 'Край,',
 'Залесовский',
 'Район,',
 'Залесово',
 'Село,',
 'автодорога',
 'Мартыново-Тогул-Залесово',
 'Территория,',
 '151-й',
 'Километр']

Проставим каждому токену тип. Для этого введем класс TypedWord

In [8]:
class TypedWord:
    word = ''
    type = ElementType.OTHER
    def __init__(self, word:str, type:ElementType):
        self.word = word
        self.type = type
    
    def __repr__(self):
        return self.word + ":" + self.type.name

### Функция конвертации набора адресо-образующих элементов в TypedWords

In [14]:
def to_typed_word_list(input , elementType :ElementType  ) -> list:
    words = list(filter(('').__ne__, input.split(' ')))
    res = list()
    for word in words:
        res.append(TypedWord(word, elementType))
    return res


def convert_address_elements(elements: dict):
    return (to_typed_word_list(elements['region_name'], ElementType.REGION) +
            to_typed_word_list(elements['region_type_name'], ElementType.REGION_TYPE) +
            to_typed_word_list(elements['area_name'], ElementType.AREA) +
            to_typed_word_list(elements['area_type_name'], ElementType.AREA_TYPE) +
            to_typed_word_list(elements['city_name'], ElementType.CITY) +
            to_typed_word_list(elements['city_type_name'], ElementType.CITY_TYPE) +
            to_typed_word_list(elements['territory_name'], ElementType.TERRITORY) +
            to_typed_word_list(elements['territory_type_name'], ElementType.TERRITORY_TYPE) +
            to_typed_word_list(elements['street_name'], ElementType.STREET) +
            to_typed_word_list(elements['street_type_name'], ElementType.STREET_TYPE)
            )

# elements = df.sample().to_dict('records')
# convert_address_elements(elements[0])

In [16]:
sample_typed_words = convert_address_elements(df.sample().to_dict('records')[0])
sample_typed_words

[Псковская:REGION,
 Область:REGION_TYPE,
 Великолукский:AREA,
 Район:AREA_TYPE,
 Готрово:CITY,
 Деревня:CITY_TYPE]

# TypedWord list to tokens and ner_tags

In [18]:
class AddressTransform:
    # p - is probability to apply transform
    def __init__(self, p):
        self.proba = p

    def transform(self, input) -> list:
        return None

In [23]:
class SimpleAddressTransform(AddressTransform):
    def transform(self, input:list) -> tuple:
        words = []
        type_codes = []
        for element in input:
            words.append(element.word)
            type_codes.append(element.type.value)
        return words, type_codes       

In [24]:
transform = SimpleAddressTransform(1)
transform.transform(sample_typed_words)

(['Псковская', 'Область', 'Великолукский', 'Район', 'Готрово', 'Деревня'],
 [1, 2, 3, 4, 7, 8])

In [28]:
def typed_words_to_words(input:list):
    words = []
    for element in input:
        words.append(element.word)
    return words    

In [29]:
def typed_words_to_type_codes(input:list):
    type_codes = []
    for element in input:
        type_codes.append(element.type.value)
    return type_codes    

## Appply to dataframe

In [32]:
df['typed_words'] = df.apply(convert_address_elements, axis=1)
df['words'] = df.apply(lambda row: typed_words_to_words(row.typed_words),  axis=1)
df['type_codes'] = df.apply(lambda row: typed_words_to_type_codes(row.typed_words),  axis=1)

In [33]:
df

,region_name,region_type_name,region_type_name_short,area_name,area_type_name,area_type_name_short,city_name,city_type_name,city_type_name_short,territory_name,territory_type_name,territory_type_name_short,street_name,street_type_name,street_type_name_short,typed_words,words,type_codes
0,Адыгея,Республика,Респ,,,,,,,,,,,,,"[Адыгея:REGION, Республика:REGION_TYPE]","[Адыгея, Республика]","[1, 2]"
1,Башкортостан,Республика,Респ,,,,,,,,,,,,,"[Башкортостан:REGION, Республика:REGION_TYPE]","[Башкортостан, Республика]","[1, 2]"
2,Алтай,Республика,Респ,,,,,,,,,,,,,"[Алтай:REGION, Республика:REGION_TYPE]","[Алтай, Республика]","[1, 2]"
3,Кабардино-Балкарская,Республика,Респ,,,,,,,,,,,,,"[Кабардино-Балкарская:REGION, Республика:REGIO...","[Кабардино-Балкарская, Республика]","[1, 2]"
4,Калмыкия,Республика,Респ,,,,,,,,,,,,,"[Калмыкия:REGION, Республика:REGION_TYPE]","[Калмыкия, Республика]","[1, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519314,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Малиновая,Улица,ул.,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор...","[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"
1519315,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Персиковая,Улица,ул.,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор...","[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"
1519316,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Яблоневая,Улица,ул.,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор...","[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"
1519317,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Садовая,Улица,ул.,"[Краснодарский:REGION, Край:REGION_TYPE, Тихор...","[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"


In [38]:
df_subset = df[['words','type_codes']]

In [39]:
df_subset

,words,type_codes
0,"[Адыгея, Республика]","[1, 2]"
1,"[Башкортостан, Республика]","[1, 2]"
2,"[Алтай, Республика]","[1, 2]"
3,"[Кабардино-Балкарская, Республика]","[1, 2]"
4,"[Калмыкия, Республика]","[1, 2]"
...,...,...
1519314,"[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"
1519315,"[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"
1519316,"[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"
1519317,"[Краснодарский, Край, Тихорецкий, Район, Тихор...","[1, 2, 3, 4, 7, 8, 5, 6, 6, 9, 10]"


## В формате Datasets

In [40]:
dataset = Dataset.from_pandas(df_subset)

In [41]:
train_testvalid = dataset.train_test_split(test_size=0.33, seed=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']})

In [42]:
dataset

DatasetDict({
    train: Dataset({
        features: ['words', 'type_codes'],
        num_rows: 1017943
    })
    validation: Dataset({
        features: ['words', 'type_codes'],
        num_rows: 250688
    })
    test: Dataset({
        features: ['words', 'type_codes'],
        num_rows: 250688
    })
})

In [43]:
dataset.save_to_disk('address_elements')

Saving the dataset (0/1 shards):   0%|          | 0/1017943 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/250688 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/250688 [00:00<?, ? examples/s]